In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from typing import TypedDict,Literal
from pydantic import BaseModel,Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [2]:
model=ChatOllama(model='gemma3')


In [3]:
class Sentiment_Schema(BaseModel):
    sentiment:Literal['positive','negative']=Field(description='Sentiment of the review')

In [4]:
parser=PydanticOutputParser(pydantic_object=Sentiment_Schema)


In [5]:
review="The software is so good"


In [6]:
class review_state(TypedDict):
    review:str
    sentiment:Literal["positive","negative"]
    diagnosis:dict
    response:str

In [7]:
def find_sentiment(state:review_state):
    template=ChatPromptTemplate.from_template('Give the sentiment of the following review:{review}\n{format_instruction}',
                                          
                              partial_variables={'format_instruction':parser.get_format_instructions()})
    prompt=template.format(review=state['review'])
    result=model.invoke(prompt)
    final_result=parser.parse(result.content)
    return {'sentiment':final_result.sentiment}


def check_sentiment(state:review_state)->Literal["positive_response","run_diagnosis"]:
    if state['sentiment']=='postive':
        return 'positive_response'
    else:
        return 'run_diagnosis'    

In [8]:
def positive_response(state:review_state):
     prompt=f"Write a warm thank-you response  for the review:{state['review']}"
     result=model.invoke(prompt)
     
     response=result.content
     return {'response':response}
 


In [9]:
class Diagnosis(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [10]:
parser_1=PydanticOutputParser(pydantic_object=Diagnosis)

In [11]:
def run_diagnosis(state:review_state):
     template=ChatPromptTemplate.from_template('Diagnose this negative review:{review}\n"Return the issue_type,tone,urgency"\n{format_instruction}',                                          
     partial_variables={'format_instruction':parser_1.get_format_instructions()})   
     
     prompt=template.format(review=state['review'])
     result=model.invoke(prompt)
     final_result=parser_1.parse(result.content)
     print(final_result)
     return {'diagnosis':final_result.model_dump()}
     
def negative_response(state:review_state):
    diagnosis=state['diagnosis']
    prompt=f"""You are a support assistant.
      The user had a '{diagnosis['issue_type']}'issue,sounded '{diagnosis['tone']}',and marked urgency '{diagnosis['urgency']}'.
      Write an empathetic,helpful resolution message."""
      
    response=model.invoke(prompt)
    return {'response':response.content}
                                              
                                              
                                              

In [12]:
graph=StateGraph(review_state)

#adding nodes

graph.add_node('find_sentiment',find_sentiment)
graph.add_node('positive_response',positive_response)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)


#adding edge
graph.add_edge(START,'find_sentiment')
graph.add_conditional_edges('find_sentiment',check_sentiment)
graph.add_edge('positive_response',END)

graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response',END)

wf=graph.compile()



In [13]:
intial_state={
    'review':"I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}
wf.invoke(intial_state)

issue_type='Bug' tone='frustrated' urgency='high'


{'review': 'I’ve been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.',
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'},
 'response': 'Okay, here\'s a draft resolution message designed to be empathetic, helpful, and address a high-urgency "Bug" issue:\n\nSubject: Regarding Your Urgent Bug Report - Let\'s Get This Fixed!\n\nDear [User Name],\n\nI’m so sorry to hear you’re experiencing this frustrating bug – I completely understand how disruptive it can be, and I sincerely apologize for the trouble it’s causing.  We know your time is valuable, and we appreciate you bringing this to our attention so quickly. \n\nWe\'ve marked this as a high-priority issue, and our team is actively investigating the root cause. [ Briefly state what’s being done - e.g., "Our develope